# Analyze 2018 Pilot runs in Parallel

## Shell Variables
Assign the variables in this notebook.

In [ ]:
source stress_test_config.sh

### Running the full pipeline on all the samples

In [ ]:
trim_func() {
    # echo $FASTQ
    FASTQ=$1
    FASTQ=${FASTQ##*/} # strip directory from file path
    # echo $FASTQ
    SAMPLE="${FASTQ%_L001_R1_001.fastq.gz}" # strip .fq.gz file extension
    echo $SAMPLE
    # exit 1
    
    /usr/bin/time -v --format='(%Xtext+%Ddata %Mmax)' STAR \
    --runMode alignReads \
    --runThreadN $THREADS \
    --readFilesCommand zcat \
    --genomeDir $GENOME_DIR \
    --outSAMtype BAM SortedByCoordinate \
    --quantMode GeneCounts \
    --genomeLoad LoadAndKeep \
    --twopassMode None \
    --limitBAMsortRAM 1000000000 \
    --outFileNamePrefix ${STAR_OUT}/${SAMPLE} \
    --outFilterScoreMinOverLread 0 --outFilterMatchNminOverLread 0 \
    --outFilterMatchNmin 0 --outFilterMismatchNmax 2 \
    --readFilesIn $TRIMMED/${SAMPLE}.trim.fastq.gz
}
export -f trim_func


STAR --genomeDir $GENOME_DIR \
    --genomeLoad LoadAndExit 

# parallel --jobs 1 pipeline_func ::: $RAW_FASTQS/1_MA_*_L001_R1_001.fastq.gz
# parallel --jobs $MAX_JOBS trim_func ::: $RAW_FASTQS/35_MA_*_L001_R1_001.fastq.gz
# parallel --jobs $MAX_JOBS trim_func ::: $RAW_FASTQS/*_L001_R1_001.fastq.gz
parallel --jobs $MAX_JOBS trim_func ::: $RAW_FASTQS/[3-4]_RZ_*_L001_R1_001.fastq.gz
    
STAR --genomeDir $GENOME_DIR \
    --genomeLoad Remove 
